<center>
<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-11t1vbu4x-xkBIHmOREQnYnYDH1GDfCg">Slack Community</a>
    </p>
</center>

# **Arize Agent Mastry Course: Tools**

In this lab, we will explore the concept of tools, which are external functions or APIs that extend an agent’s capabilities beyond basic language understanding. Tools allow agents to access real-time information, perform calculations, query databases, and interact with specialized systems.

Dedicated tools are important because they enable agents to provide accurate, context-aware, and actionable responses instead of relying only on general reasoning. By connecting an agent to focused data sources or APIs, we can significantly improve the quality, relevance, and reliability of its outputs.

We will build on our existing agent from previous labs and focus on improving its tools. In particular, we will enhance their functionality with more targeted APIs to help the agent deliver stronger, more grounded answers.

https://youtu.be/X95MFcYH1_s

best definition of RAG and MCP

# Set up

In [ ]:
!pip install -qqqqqq arize-otel agno openai openinference-instrumentation-agno openinference-instrumentation-openai openinference-instrumentation-mcp httpx

In [ ]:
# import os
# from getpass import getpass
#
# os.environ["ARIZE_SPACE_ID"] = globals().get("ARIZE_SPACE_ID") or getpass("🔑 Enter your Arize Space ID: ")
#
# os.environ["ARIZE_API_KEY"] = globals().get("ARIZE_API_KEY") or getpass("🔑 Enter your Arize API Key: ")
#
# os.environ["OPENAI_API_KEY"] = globals().get("OPENAI_API_KEY") or getpass("🔑 Enter your OpenAI API Key: ")
#
# os.environ["TAVILY_API_KEY"] = globals().get("TAVILY_API_KEY") or getpass("🔑 Enter your Tavily API Key: ")

In [1]:
OPENAI_API_KEY="k-proj-_nS-GmcG9QDvjU3zVmRtBpGVDFj5oh3ijinzHyCNLhNiBpu-6xBsKIylI4TcYsrArtb7v5RUKmT3BlbkFJtdVGae3sJN2aol43xpmy3gZJViFiXEn8iL1AvjrM8OedEhKCtMQIpKYRZB1e_BMtdiQ-TJF5MA"
# Or use OpenRouter (OpenAI-compatible)
# OPENROUTER_API_KEY=your_openrouter_api_key_here
# OPENROUTER_MODEL=openai/gpt-4o-mini

# Observability with Arize (https://app.arize.com)
ARIZE_SPACE_ID="U3BhY2U6NjQwOnpLb2k="
ARIZE_API_KEY="ak-8f30dd6d-75f7-4aa7-9e02-2dfe410d7693-30xLYLjC3k7TPvOd_qHulh9ut3Ga6wyo"

In [2]:
import os
import httpx
from dotenv import load_dotenv
from getpass import getpass
import httpx
from urllib.parse import quote
from typing import Optional

os.environ["ARIZE_SPACE_ID"] = ARIZE_SPACE_ID
os.environ["ARIZE_API_KEY"] = ARIZE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [45]:
from arize.otel import register
from openinference.instrumentation.openai import OpenAIInstrumentor
from openinference.instrumentation.agno import AgnoInstrumentor

model_id = "travel-agent-demo"
tracer_provider = register(
    space_id=os.getenv("ARIZE_SPACE_ID"),
    api_key=os.getenv("ARIZE_API_KEY"),
    project_name=model_id,
    set_global_tracer_provider=True
)
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)
AgnoInstrumentor().instrument(tracer_provider=tracer_provider)

OpenTelemetry Tracing Details
|  Arize Project: travel-agent-demo
|  Span Processor: BatchSpanProcessor
|  Collector Endpoint: otlp.arize.com
|  Transport: gRPC
|  Transport Headers: {'authorization': '****', 'api_key': '****', 'arize-space-id': '****', 'space_id': '****', 'arize-interface': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



# Let's redefine the tools our agent uses

Instead of relying on a general search API, we can define dedicated API requests so that each tool delivers more precise and relevant results.

To enhance the performance of our essential_info tool, we’ll integrate specialized APIs: a weather API for real-time conditions and Wikipedia Search for accurate, up-to-date contextual information.

These functions are marked with decorators to ensure they are explicitly traced during the agent’s execution.

In [46]:
# --- Helper functions for tools ---
import httpx
from opentelemetry import trace

tracer = trace.get_tracer(__name__)

# @tracer.chain(name="search-api")
def _search_api(query: str) -> str | None:
    """Try Tavily search first, fall back to None."""
    tavily_key = os.getenv("TAVILY_API_KEY")
    if not tavily_key:
        return None
    try:
        resp = httpx.post(
            "https://api.tavily.com/search",
            json={
                "api_key": tavily_key,
                "query": query,
                "max_results": 3,
                "search_depth": "basic",
                "include_answer": True,
            },
            timeout=8,
        )
        data = resp.json()
        answer = data.get("answer") or ""
        snippets = [r.get("content", "") for r in data.get("results", [])]
        combined = " ".join([answer] + snippets).strip()
        return combined[:400] if combined else None
    except Exception:
        return None

def _compact(text: str, limit: int = 200) -> str:
    """Compact text for cleaner outputs."""
    cleaned = " ".join(text.split())
    return cleaned if len(cleaned) <= limit else cleaned[:limit].rsplit(" ", 1)[0]


In [47]:
# dir(tracer)
@tracer.chain(name='api-name')
def _compact(text: str, limit: int = 200) -> str:
    """Compact text for cleaner outputs."""
    cleaned = " ".join(text.split())
    return cleaned if len(cleaned) <= limit else cleaned[:limit].rsplit(" ", 1)[0]

In [4]:
encoded_dest = quote('Dubai')
url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{encoded_dest}"
HEADERS = { 'User-Agent': 'MyArizeApp/1.0 (ExampleContac@example.com)'}

try:
    r = httpx.get(url, headers = HEADERS, timeout=5)
    r.raise_for_status()

    data = r.json().get("extract")
except httpx.HTTPStatusError as e:
    if e.response.status_code == 404:
    # except httpx.RequestError as e:
        print(e)

In [21]:
cleaned[:200].rsplit(" ", 1)[0]

'Dubai is the most populous city in the United Arab Emirates and the capital of the Emirate of Dubai. It is on a creek on the southeastern coast of the Persian Gulf. As of 2025, its population stands'

In [16]:
cleaned = " ".join(data.split())
if len(cleaned) <= 50:
    print(cleaned)
else:
    cleaned[:50].rsplit(" ", 1)[0]
    print(f"version: {cleaned}")

version: Dubai is the most populous city in the United Arab Emirates and the capital of the Emirate of Dubai. It is on a creek on the southeastern coast of the Persian Gulf. As of 2025, its population stands at 4 million, 92% of whom are expatriates. The wider urban area includes Sharjah and has a population of 5 million people as of 2023, while the Dubai–Sharjah–Ajman metropolitan area has a population of 6 million people.


In [5]:
data

'Dubai is the most populous city in the United Arab Emirates and the capital of the Emirate of Dubai. It is on a creek on the southeastern coast of the Persian Gulf. As of 2025, its population stands at 4 million, 92% of whom are expatriates. The wider urban area includes Sharjah and has a population of 5 million people as of 2023, while the Dubai–Sharjah–Ajman metropolitan area has a population of 6 million people.'

In [29]:
# APIs for Essential Info Tool


# @tracer.chain(name="wiki-summary-api")
def _wiki_summary(dest: str) -> str:
    if not dest:
        return ""
    encoded_dest = quote(dest)

    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{encoded_dest}"
    HEADERS = { 'User-Agent': 'MyArizeApp/1.0 (ExampleContac@example.com)'}

    try:
        r = httpx.get(url, headers = HEADERS, timeout=5)
        r.raise_for_status()

        data = r.json().get("extract")
        return data if data else ""

    except httpx.HTTPStatusError as e:
        if e.response.status_code == 404:
            return ""
        return ""
    except httpx.RequestError as e:
        return ""
    except Exception as e:
        return ""

# @tracer.chain(name="weather-api")
def _weather(dest):
    g = httpx.get(f"https://geocoding-api.open-meteo.com/v1/search?name={dest}")
    if g.status_code != 200 or not g.json().get("results"):
        return ""
    lat, lon = g.json()["results"][0]["latitude"], g.json()["results"][0]["longitude"]
    w = httpx.get(f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true").json()
    cw = w.get("current_weather", {})
    return f"Weather now: {cw.get('temperature')}°C, wind {cw.get('windspeed')} km/h."

In [30]:
_weather('Dubai')

'Weather now: 23.2°C, wind 15.3 km/h.'

In [ ]:
from agno.tools import tool

@tool
def essential_info(destination: str) -> str:
    """Get essential info (summary and weather) using APIs"""
    parts = []
    wiki = _wiki_summary(destination)
    if wiki: parts.append(wiki)
    weather = _weather(destination)
    if weather: parts.append(weather)
    return f"{destination} essentials:\n" + "\n".join(parts)

@tool
def budget_basics(destination: str, duration: str) -> str:
    """Summarize travel cost categories."""
    q = f"{destination} travel budget average daily costs {duration}"
    s = _search_api(q)
    if s:
        return f"{destination} budget ({duration}): {_compact(s)}"
    return f"Budget for {duration} in {destination} depends on lodging, meals, transport, and attractions."


@tool
def local_flavor(destination: str, interests: str = "local culture") -> str:
    """Suggest authentic local experiences."""
    q = f"{destination} authentic local experiences {interests}"
    s = _search_api(q)
    if s:
        return f"{destination} {interests}: {_compact(s)}"
    return f"Explore {destination}'s unique {interests} through markets, neighborhoods, and local eateries."


# Define and Run Agent

We define and run the agent just as we did in the first lab. The key difference lies in the quality of the output, which improves thanks to the richer, more accurate information our tool can now retrieve.

In [ ]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat

# --- Main Agent ---
trip_agent = Agent(
    name="TripPlanner",
    role="AI Travel Assistant",
    model=OpenAIChat(id="gpt-4.1"),
    instructions=(
        "You are a friendly and knowledgeable travel planner. "
        "Combine multiple tools to create a trip plan including essentials, budget, and local flavor. "
        "Keep the tone natural, clear, and under 1000 words."
    ),
    markdown=True,
    tools=[essential_info, budget_basics, local_flavor],
)

In [ ]:
# --- Example usage ---
destination = "Tokyo"
duration = "5 days"
interests = "food, culture"

query = f"""
Plan a {duration} trip to {destination}.
Focus on {interests}.
Include essential info, budget breakdown, and local experiences.
"""
trip_agent.print_response(
  query,
  stream=True
)

View the trace in Arize! You can see the API calls nested under each tool span.

![Results](https://storage.googleapis.com/arize-phoenix-assets/assets/images/arize-course-lab4.png)